In [1]:
from __future__ import division, print_function
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '4'

#Purpose of this notebook is to replicate ExPecto chromatin features

#start by reading in the .npy features
npy_features_file = "/srv/scratch/avanti/ExPecto/resources/Xreducedall.2002.npy"
expecto_features = np.load(npy_features_file)

In [15]:
#now read in the gene file
gene_file = "/srv/scratch/avanti/ExPecto/resources/geneanno.csv"
gene_chrom_tss_strand = []
for i,line in enumerate(open(gene_file)):
    gene_id,symbol,chrom,strand,TSS,CAGE_TSS,gene_type = line.rstrip().split(",")
    if (i > 0):
        gene_chrom_tss_strand.append((gene_id, chrom, int(CAGE_TSS), (1 if strand=="+" else -1)) )
    

In [34]:
#now load the pytorch model
import torch
from torch.utils.serialization import load_lua

model = load_lua('/srv/scratch/avanti/ExPecto/resources/deepsea.beluga.2002.cpu')
model.evaluate()
model.cuda()

nn.Sequential {
  [input -> (0) -> (1) -> (2) -> output]
  (0): nn.Sequential {
    [input -> (0) -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> output]
    (0): nn.SpatialConvolution(4 -> 320, 1x8)
    (1): nn.ReLU
    (2): nn.SpatialConvolution(320 -> 320, 1x8)
    (3): nn.ReLU
    (4): nn.Dropout(0.2000)
    (5): nn.SpatialMaxPooling(1x4, 1, 4)
    (6): nn.SpatialConvolution(320 -> 480, 1x8)
    (7): nn.ReLU
    (8): nn.SpatialConvolution(480 -> 480, 1x8)
    (9): nn.ReLU
    (10): nn.Dropout(0.2000)
    (11): nn.SpatialMaxPooling(1x4, 1, 4)
    (12): nn.SpatialConvolution(480 -> 640, 1x8)
    (13): nn.ReLU
    (14): nn.SpatialConvolution(640 -> 640, 1x8)
    (15): nn.ReLU
  }
  (1): nn.Sequential {
    [input -> (0) -> (1) -> (2) -> (3) -> (4) -> output]
    (0): nn.Dropout(0.5000)
    (1): nn.Reshape(67840)
    (2): nn.Linear(67840 -> 2003)
    (3): nn.ReLU
    (4): nn.Linear(2003 -> 2002)
  }
  (2): nn.Sigmoid
}

In [35]:
import math

def encodeSeqs(seqs, inputsize=2000):
    """Convert sequences to 0-1 encoding and truncate to the input size.
    The output concatenates the forward and reverse complement sequence
    encodings.
    Args:
        seqs: list of sequences (e.g. produced by fetchSeqs)
        inputsize: the number of basepairs to encode in the output
    Returns:
        numpy array of dimension: (2 x number of sequence) x 4 x inputsize
    2 x number of sequence because of the concatenation of forward and reverse
    complement sequences.
    """
    seqsnp = np.zeros((len(seqs), 4, inputsize), np.bool_)

    mydict = {'A': np.asarray([1, 0, 0, 0]), 'G': np.asarray([0, 1, 0, 0]),
            'C': np.asarray([0, 0, 1, 0]), 'T': np.asarray([0, 0, 0, 1]),
            'N': np.asarray([0, 0, 0, 0]), 'H': np.asarray([0, 0, 0, 0]),
            'a': np.asarray([1, 0, 0, 0]), 'g': np.asarray([0, 1, 0, 0]),
            'c': np.asarray([0, 0, 1, 0]), 't': np.asarray([0, 0, 0, 1]),
            'n': np.asarray([0, 0, 0, 0]), '-': np.asarray([0, 0, 0, 0])}

    n = 0
    for line in seqs:
        cline = line[int(math.floor(((len(line) - inputsize) / 2.0))):int(math.floor(len(line) - (len(line) - inputsize) / 2.0))]
        for i, c in enumerate(cline):
            seqsnp[n, :, i] = mydict[c]
        n = n + 1

    # get the complementary sequences
    #dataflip = seqsnp[:, ::-1, ::-1]
    #seqsnp = np.concatenate([seqsnp, dataflip], axis=0)
    return seqsnp.astype("float32")

In [235]:
import pyfasta

hg19_path = "/mnt/data/annotations/by_organism/human/hg19.GRCh37/hg19.genome.fa"
genome = pyfasta.Fasta(hg19_path)

windowsize = 2000
predictions_fwdonly = []
predictions_withrc = []
maxshift=20000
shifts = np.array(list(range(-20000,20000,200)))+100
print("shifts:\n",shifts)
assert len(shifts)==200
for gene,chrom,tss,strand in gene_chrom_tss_strand[:3]:
    seqs_to_predict = []
    for shift in shifts:
        seq = genome.sequence({'chr': chrom, 'start': tss + shift*strand -
                               int(0.5*windowsize - 1), 'stop': tss + shift*strand + int(0.5*windowsize)})
        seqs_to_predict.append(seq)

    seqsnp = encodeSeqs(seqs_to_predict)

    model_input = torch.from_numpy(np.array(seqsnp)).unsqueeze(3)
    rc_model_input = torch.from_numpy(np.array(seqsnp[:,::-1,::-1])).unsqueeze(3)
    model_input = model_input.cuda()
    rc_model_input = rc_model_input.cuda()
    prediction = model.forward(model_input).cpu().numpy().copy()
    rc_prediction = model.forward(rc_model_input).cpu().numpy().copy()
    predictions_fwdonly.append(prediction)
    predictions_withrc.append(0.5*(prediction+rc_prediction))

predictions_fwdonly=np.array(predictions_fwdonly)
predictions_withrc=np.array(predictions_withrc)

shifts:
 [-19900 -19700 -19500 -19300 -19100 -18900 -18700 -18500 -18300 -18100
 -17900 -17700 -17500 -17300 -17100 -16900 -16700 -16500 -16300 -16100
 -15900 -15700 -15500 -15300 -15100 -14900 -14700 -14500 -14300 -14100
 -13900 -13700 -13500 -13300 -13100 -12900 -12700 -12500 -12300 -12100
 -11900 -11700 -11500 -11300 -11100 -10900 -10700 -10500 -10300 -10100
  -9900  -9700  -9500  -9300  -9100  -8900  -8700  -8500  -8300  -8100
  -7900  -7700  -7500  -7300  -7100  -6900  -6700  -6500  -6300  -6100
  -5900  -5700  -5500  -5300  -5100  -4900  -4700  -4500  -4300  -4100
  -3900  -3700  -3500  -3300  -3100  -2900  -2700  -2500  -2300  -2100
  -1900  -1700  -1500  -1300  -1100   -900   -700   -500   -300   -100
    100    300    500    700    900   1100   1300   1500   1700   1900
   2100   2300   2500   2700   2900   3100   3300   3500   3700   3900
   4100   4300   4500   4700   4900   5100   5300   5500   5700   5900
   6100   6300   6500   6700   6900   7100   7300   7500   7700   79

In [236]:
pos_weight_shifts = shifts
pos_weights = np.vstack([
        np.exp(-0.01*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts <= 0),
        np.exp(-0.02*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts <= 0),
        np.exp(-0.05*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts <= 0),
        np.exp(-0.1*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts <= 0),
        np.exp(-0.2*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts <= 0),
        np.exp(-0.01*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts >= 0),
        np.exp(-0.02*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts >= 0),
        np.exp(-0.05*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts >= 0),
        np.exp(-0.1*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts >= 0),
        np.exp(-0.2*np.abs(pos_weight_shifts)/200)*(pos_weight_shifts >= 0)])

In [237]:
reconstructed_expecto_fwdonly = np.sum(pos_weights[None,:,:,None]*predictions_fwdonly[:,None,:,:],axis=2)
reconstructed_expecto_withrc = np.sum(pos_weights[None,:,:,None]*predictions_withrc[:,None,:,:],axis=2)

In [238]:
print(reconstructed_expecto_fwdonly[0,:,0])
print(reconstructed_expecto_withrc[0,:,0])

[1.35015494 1.03121655 0.69268325 0.58555646 0.53042333 0.52554336
 0.47175934 0.39041655 0.32987617 0.26208855]
[1.35120312 1.02925651 0.68654915 0.57727376 0.52144186 0.57660481
 0.51948136 0.42981128 0.35950798 0.28117582]


In [239]:
expecto_features[0].reshape(10,2002)[:,0]

array([1.22746855, 0.91435068, 0.5421533 , 0.41338246, 0.36833563,
       0.67654061, 0.62398969, 0.54676042, 0.49142184, 0.43067582])

In [240]:
from scipy.stats import spearmanr
print(spearmanr(reconstructed_expecto_fwdonly.flatten(), expecto_features[:len(reconstructed_expecto)].flatten()))
print(spearmanr(reconstructed_expecto_withrc.flatten(), expecto_features[:len(reconstructed_expecto)].flatten()))

SpearmanrResult(correlation=0.9900563526481763, pvalue=0.0)
SpearmanrResult(correlation=0.9918444705437338, pvalue=0.0)
